In [1]:
import numpy as np
import pandas as pd

1) Даны значения величины заработной платы заемщиков банка (zp) 
и значения их поведенческого кредитного скоринга (ks):

    zp = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110],
    ks = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832].

Используя математические операции, посчитать коэффициенты линейной регрессии,
приняв за X заработную плату (то есть, zp - признак), 
а за y - значения скорингового балла (то есть, ks - целевая переменная).
Произвести расчет как с использованием intercept, так и без.


In [2]:
%%latex
Коэффициенты уравнения линейной регрессии ($y=a + b \cdot x$) можно найти следующим образом:

$b = \frac{\overline{yx} - \overline{y} \cdot {\overline{x}}}{\overline{x^2} - (\overline{x})^2};\\
a = \overline{y} - b \cdot {\overline{x}}.\\
\text{Или матричным методом для уравнения }y = \beta_0 + \beta_1\cdot x\\
\begin{pmatrix}
y_1\\ y_2\\ \ldots \\ y_n\\
\end{pmatrix} = \begin{pmatrix}
1 & x_1\\ 1 & x_2 \\\ldots & \ldots \\ 1 & y_n
\end{pmatrix} \begin{pmatrix}
\beta_0\\ \beta_1
\end{pmatrix}\\
\hat{B} = (X^T\cdot X)^{-1}\cdot X^T\cdot Y\\
\text{Коэффициент с предположением функции модели как: } y=\beta_1 \cdot x\\
\begin{pmatrix}
y_1\\ y_2\\ \ldots \\ y_n\\
\end{pmatrix} = \begin{pmatrix}
x_1\\ x_2\\ \ldots \\ y_n\\
\end{pmatrix} \cdot \beta_1\\
\hat{B} = (X^T\cdot X)^{-1}\cdot X^T\cdot Y$

<IPython.core.display.Latex object>

In [3]:
zp = np.array([35, 45, 190, 200, 40, 70, 54, 150, 120, 110]) 
ks = np.array([401, 574, 874, 919, 459, 739, 653, 902, 746, 832])
zp_ks = ks*zp
n= len(zp)
b = (np.mean(zp_ks) - np.mean(ks)*np.mean(zp))/(np.mean(zp**2)-(np.mean(zp))**2) 
b_ = (n*(np.sum(zp_ks)) - (np.sum(zp) * np.sum(ks)))/(n*(np.sum(zp**2)) - (np.sum(zp)**2))
a = np.mean(ks) - b* np.mean(zp)
y_hat = a + b*zp
df = pd.DataFrame({'X(zp)': zp, 'Y(ks)': ks, 'y_hat': y_hat, '(y_hat - Y(ks))^2': (y_hat - ks)**2}, columns=['X(zp)', 'Y(ks)', 'y_hat', '(y_hat - Y(ks))^2'])


zp_without_intercept = zp.reshape((10, 1))
zp_with_intercept = np.hstack([np.ones((10, 1)), zp_without_intercept])
ks_ = ks.reshape((10, 1))

B_hat_with_intercept = np.dot(np.linalg.inv(np.dot(zp_with_intercept.T, zp_with_intercept)), zp_with_intercept.T @ ks_)
B_hat_without_intercept = np.dot(np.linalg.inv(np.dot(zp_without_intercept.T, zp_without_intercept)), zp_without_intercept.T @ ks_)

print(f'{df} \n\n'
      f' y = {a} + {b}*x,\n'
      f'((Y(ks)-y_hat)^2.sum()) / n: {(((y_hat - ks)**2).sum()/n)},\n'
      f' without_intercept: y = {B_hat_without_intercept[0, 0]}*x\n'
      f' with_intercept y = 𝛽0 + 𝛽1*x : y = {B_hat_with_intercept[0, 0]} + {B_hat_with_intercept[1, 0]}*x')

   X(zp)  Y(ks)       y_hat  (y_hat - Y(ks))^2
0     35    401  535.896218       18196.989687
1     45    574  562.101607         141.571755
2    190    874  942.079745        4634.851677
3    200    919  968.285134        2429.024414
4     40    459  548.998913        8099.804273
5     70    739  627.615079       12406.600606
6     54    653  585.686457        4531.113075
7    150    902  837.258190        4191.502003
8    120    746  758.642023         159.820751
9    110    832  732.436634        9912.863772 

 y = 444.1773573243596 + 2.620538882402765*x,
((Y(ks)-y_hat)^2.sum()) / n: 6470.414201176658,
 without_intercept: y = 5.889820420132688*x
 with_intercept y = 𝛽0 + 𝛽1*x : y = 444.1773573243595 + 2.6205388824027693*x


2 )Посчитать коэффициент линейной регрессии при заработной плате (zp),
используя градиентный спуск (без intercept).


In [4]:
def mse_without_intercept(B1, y=ks, X=zp, n=10):
    return np.sum((B1*X - y)**2)/n


In [5]:
B1 = 5

alpha = 1e-6

for i in range(10000):
    B1 -= alpha*(2/n)*np.sum((B1*zp - ks)*zp)
    if i%3000==0:
        print(f'Итерация {i}, B1 = {B1}, mse = {mse_without_intercept(B1)}')



Итерация 0, B1 = 5.0245174, mse = 66832.07825149018
Итерация 3000, B1 = 5.889820420132673, mse = 56516.85841571943
Итерация 6000, B1 = 5.889820420132673, mse = 56516.85841571943
Итерация 9000, B1 = 5.889820420132673, mse = 56516.85841571943


3) В каких случаях для вычисления доверительных интервалов и проверки статистических гипотез используется таблица значений функции Лапласа, а в каких - таблица критических точек распределения Стьюдента?
    
    Таблица значений функции Лапласа используется в случаях, когда  𝜎(cигма) генеральной совокупности ИЗВЕСТНА

    Таблица критических точек распределения Стьюдент используется, если 𝜎(cигма) генеральной совокупности НЕ ИЗВЕСТНА


4 ) Произвести вычисления как в пункте 2, но с вычислением intercept. 
Учесть, что изменение коэффициентов должно производиться на каждом шаге одновременно

(то есть изменение одного коэффициента не должно влиять на изменение другого во время одной итерации).

In [6]:
def mse_with_intercept(B0, B1, y=ks, X=zp, n=10):
    return np.sum((B0 + B1*X - y)**2)/n

B0 = 0.1
B1 = 0.1

alpha = 1e-6
for i in range(5000000): 
    y_hat = B0 + B1*zp
    B0 -= alpha*(2/n)*np.sum((y_hat - ks))
    B1 -= alpha*(2/n)*np.sum((y_hat - ks)*zp)
    if i%500000==0:
        print(f'Итерация {i}, B0 = {B0}, B1 = {B1}, mse = {mse_with_intercept(B0, B1)}')

Итерация 0, B0 = 0.10139932, B1 = 0.2595078, mse = 493102.2473380378
Итерация 500000, B0 = 99.62326925415903, B1 = 5.156609306712451, mse = 36584.8831651681
Итерация 1000000, B0 = 176.81634536031407, B1 = 4.588434137551724, mse = 24602.866672921762
Итерация 1500000, B0 = 236.71526880642702, B1 = 4.147551578153019, mse = 17388.28338336958
Итерация 2000000, B0 = 283.1945803067062, B1 = 3.8054432958269095, mse = 13044.25564183225
Итерация 2500000, B0 = 319.2607777147802, B1 = 3.5399801371216335, mse = 10428.639983674579
Итерация 3000000, B0 = 347.2467902759279, B1 = 3.33399071134467, mse = 8853.731700793858
Итерация 3500000, B0 = 368.96288333142047, B1 = 3.1741506652199787, mse = 7905.451668998496
Итерация 4000000, B0 = 385.8137546931221, B1 = 3.0501208017996615, mse = 7334.47555027671
Итерация 4500000, B0 = 398.88939702545025, B1 = 2.9538782931601695, mse = 6990.680766858188
